<a href="https://colab.research.google.com/github/rupeshgyawali/federated-covid-xray-detection/blob/main/federated/sequential_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Download and unzip dataset

In [ ]:
!cp /content/drive/MyDrive/MajorProject/Dataset_NIID.zip /content/

In [ ]:
!unzip -q Dataset_NIID.zip -d Dataset_NIID

### Install necessary libraries

In [ ]:
!pip install flwr

     |████████████████████████████████| 106 kB 15.0 MB/s 
     |████████████████████████████████| 1.0 MB 66.7 MB/s 
     |████████████████████████████████| 15.3 MB 50.2 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0+zzzcolab20220506162203 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
xarray-einstats 0.2.2 requires numpy>=1.21, but you have numpy 1.20.3 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imga

### Necessary imports


In [ ]:
import numpy as np
import flwr as fl
import tensorflow as tf

In [ ]:
batch_size = 32
img_height = 224
img_width = 224

### Dataset preparation

In [ ]:
def get_dataset(dataset_path):
    train_ds = tf.keras.utils.image_dataset_from_directory(
        dataset_path, seed=123, validation_split=0.2, subset="training", batch_size=batch_size, image_size=(img_height, img_width))
    test_ds = tf.keras.utils.image_dataset_from_directory(
        dataset_path, seed=123, validation_split=0.2, subset="validation", batch_size=batch_size, image_size=(img_height, img_width))

    # Configure dataset for performance
    # AUTOTUNE = tf.data.AUTOTUNE
    # train_ds = train_ds.cache().shuffle(4500).prefetch(buffer_size=AUTOTUNE)
    # test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

    return train_ds, test_ds

### Model Preparation

In [ ]:
def get_compiled_model():
    num_classes = 4

    vgg = tf.keras.applications.vgg16.VGG16(input_shape=[img_height,img_width]  + [3], weights='imagenet', include_top=False)#, input_tensor=data_augmentation)

    for layer in vgg.layers:
        layer.trainable = False
    
    x=tf.keras.layers.Flatten()(vgg.output)
    x=tf.keras.layers.Dense(64, activation='relu')(x)
    x=tf.keras.layers.Dropout(0.2)(x)
    x=tf.keras.layers.Dense(128, activation='relu')(x)
    prediction = tf.keras.layers.Dense(4,activation='softmax')(x)

    model = tf.keras.models.Model(inputs=vgg.input, outputs=prediction)

    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  metrics=['accuracy'])

    return model

In [ ]:
model = get_compiled_model()

### Federated learning


In [ ]:
def evaluate_metrics_aggregation_fn(eval_metrics):
    # Weigh accuracy of each client by number of examples used
    accuracies = [metrics * num_examples for num_examples, metrics in eval_metrics]
    examples = [num_examples for num_examples, _ in eval_metrics]

    # Aggregate and print custom metric
    accuracy_aggregated = sum(accuracies) / sum(examples)
    return {'accuracy': accuracy_aggregated}

In [ ]:
NUM_OF_CLIENTS = 3
NUM_OF_ROUNDS = 50
LOCAL_EPOCHS = 3
CLIENT_BASE_LOG_DIR = '/content/drive/MyDrive/MajorProject/logs/experiment3/federated/client'
SERVER_BASE_LOG_DIR = '/content/drive/MyDrive/MajorProject/logs/experiment3/federated/'

In [ ]:
Datasets = []
for i in range(1, NUM_OF_CLIENTS+1):
  train, test = get_dataset('/content/Dataset_NIID/client' + str(i))
  Datasets.append((train, test)) 

Found 1982 files belonging to 4 classes.
Using 1586 files for training.
Found 1982 files belonging to 4 classes.
Using 396 files for validation.
Found 1716 files belonging to 4 classes.
Using 1373 files for training.
Found 1716 files belonging to 4 classes.
Using 343 files for validation.
Found 1847 files belonging to 4 classes.
Using 1478 files for training.
Found 1847 files belonging to 4 classes.
Using 369 files for validation.


In [ ]:
aggregated_weights = model.get_weights()

In [ ]:
for round in range(1, NUM_OF_ROUNDS+1):
  print(f'Round {round}')
  # Fit round
  fit_results = []
  for client_id in range(1, NUM_OF_CLIENTS+1):
    log_dir = CLIENT_BASE_LOG_DIR + str(client_id)
    train_ds, test_ds = Datasets[client_id-1]
    model.set_weights(aggregated_weights)
    csv_logger = tf.keras.callbacks.CSVLogger(
            log_dir + 'training.log', 
            append=True)
    print(f'Client {client_id}: Fit round')
    model.fit(train_ds, validation_data=test_ds, epochs=LOCAL_EPOCHS, callbacks=[csv_logger])
    fit_results.append((model.get_weights(), len(train_ds)))
  
  # Federated Aggregation
  aggregated_weights = fl.server.strategy.aggregate.aggregate(fit_results)
  print('Weights aggreated')
  if round % 5 == 0:
    print(f"Saving round {round} aggregated_weights...")
    np.savez(SERVER_BASE_LOG_DIR + f"round-{round}-weights.npz", *aggregated_weights)

  # Evaluation round
  evaluation_results = []
  model.set_weights(aggregated_weights)
  for client_id in range(1, NUM_OF_CLIENTS+1):
    log_dir = CLIENT_BASE_LOG_DIR + str(client_id)
    _, test_ds = Datasets[client_id-1]
    print(f'Client {client_id}: Evaluation round')
    loss, acc = model.evaluate(test_ds)
    # Log client's local evaluation loss and accuracy
    with open(log_dir + 'evaluation.log', 'a') as f:
      f.write(f'{round},{loss},{acc}\n')
    evaluation_results.append((len(test_ds), loss, acc))

  # Evaluation Aggregation
  loss_aggregated = fl.server.strategy.aggregate.weighted_loss_avg([(n_examples, l) for n_examples, l, _ in evaluation_results])
  metrics_aggregated = evaluate_metrics_aggregation_fn([(n_examples, a) for n_examples, _, a in evaluation_results])
  # Log evaluation loss and accuracy
  with open(SERVER_BASE_LOG_DIR + 'server_evaluation.log', 'a') as f:
    f.write(f'{round},{loss_aggregated},{metrics_aggregated["accuracy"]}\n')
  print('Evaluation Aggregated')


Round 1
Client 1: Fit round
Epoch 1/3
50/50 [==============================] - 11s 204ms/step - loss: 1.9679 - accuracy: 0.7024 - val_loss: 0.5952 - val_accuracy: 0.7778
Epoch 2/3
50/50 [==============================] - 11s 202ms/step - loss: 0.4967 - accuracy: 0.8386 - val_loss: 0.6220 - val_accuracy: 0.8333
Epoch 3/3
50/50 [==============================] - 11s 203ms/step - loss: 0.4063 - accuracy: 0.8663 - val_loss: 0.6381 - val_accuracy: 0.8434
Client 2: Fit round
Epoch 1/3
43/43 [==============================] - 9s 207ms/step - loss: 4.2736 - accuracy: 0.5907 - val_loss: 0.6411 - val_accuracy: 0.7697
Epoch 2/3
43/43 [==============================] - 9s 209ms/step - loss: 0.7048 - accuracy: 0.7371 - val_loss: 0.5630 - val_accuracy: 0.8251
Epoch 3/3
43/43 [==============================] - 9s 207ms/step - loss: 0.6164 - accuracy: 0.7524 - val_loss: 0.4993 - val_accuracy: 0.7988
Client 3: Fit round
Epoch 1/3
47/47 [==============================] - 10s 204ms/step - loss: 2.6024 - 